In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:10])

gsk_1EJvXF


In [20]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이") 
    major: str = Field(description="전공")  
    hobbies: List[str] = Field(description="취미") 
    goal: str = Field(description="목표")

In [21]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

In [36]:
prompt = ChatPromptTemplate.from_template(
    """
    사용자의 입력에서 다음 정보를 추출해서 JSON으로 변환하세요.
    반드시 JSON만 출력하고 다른 설명은 하지 마세요.
    {format_instructions}

    입력: {query}
    """
).partial(format_instructions=parser.get_format_instructions())


In [34]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [37]:
# 체인 구성 및 실행
query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"query": query})

print(type(output))
print(output)

<class '__main__.MovieRecommendation'>
name='김민수' age=22 major='컴퓨터공학' hobbies=['게임하기', '영화보기', '코딩'] goal='훌륭한 개발자가 되는 것'


In [38]:
# 결과 출력
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {', '.join(output.hobbies)}")
print(f"목표: {output.goal}")

이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
